In [1]:
# default_exp kapture

# kapture

> Module for working with local Kapture data: cameras, keypoints, descriptors, matches, etc.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
SAMPLE_DATASET = "/homeLocal/Github/kapture/samples/7scenes/kapture/stairs/seq-01"
# SAMPLE_DATASET = "/homeLocal/Github/LocalFeaturesApplications/SampleDataset"

In [4]:
#export
import cv2
import torch
import kornia as K
from typing import List, Union, Tuple, Optional, Dict
import matplotlib.pyplot as plt
import numpy as np
import kapture
import kapture.io.csv as csv
from scipy.spatial.transform import Rotation

def pinhole_from_kapture(dataset_path:str) -> Dict[str,K.geometry.camera.PinholeCamera]:  
    """Read Kapture sensor and rig data.
    
    It will read through the sensors.txt from Kapture and select only the PINHOLE and SIMPLE_PINHOLE
    camera models. If there is a rigs.txt file, it will use the rotation and translation vectors.
    If there are no extrinsic parameters, it will use and identity matrix.
    
    Args:
        dataset_path: Kapture dataset path.
    
    Returns:
        sensors_dict: Dict where the keys are the kapture sensors_id and the items are
                      Kornia's PinholeCamera (K.geometry.camera.PinholeCamera).  
        
    """
    tar_handlers = csv.get_all_tar_handlers(dataset_path)
    kapture_data = csv.kapture_from_dir(dataset_path)
    
    sensors = kapture_data.sensors
    rigs = kapture_data.rigs
    keys = list(sensors.keys())
    
    sensors_dict = {}
    for sensor_id in keys:
        sensor = kapture_data.sensors[sensor_id]
        sensor_name = sensor.name
        params = sensor.sensor_params
        
        if params[0] == 'PINHOLE':
            w, h, fx, fy, cx, cy = params[1:]
        elif params[0] == "SIMPLE_PINHOLE":
            w, h, f, cx, cy = params[1:]  
            fx = f
            fy = f
        else:
            continue

        w, h = int(w), int(h)
            
        intr = torch.zeros([4,4])
        intr[0, 0] = float(fx)
        intr[1, 1] = float(fy)
        intr[2, 2] = 1.
        intr[0, 2] = float(cx)
        intr[1, 2] = float(cy)
        
        if rigs:
            for key in rigs:
                if sensor_id in rigs[key]:
                    quat = rigs[key][sensor_id].r
                    r = Rotation.from_quat([quat.x, quat.y, quat.z, quat.w]).as_matrix()
                    t = rigs[key][sensor_id].t
                    
                    extr = np.zeros([4,4])
                    extr[:3, :3] = r
                    extr[:3, 3] = t[:, 0]
                    extr[3, 3] = 1
                    extr = torch.from_numpy(extr)
        else:
            extr = torch.diag(torch.ones(4))
        
        intr = intr.unsqueeze(0)
        extr = extr.unsqueeze(0)
        w = torch.tensor([w])
        h = torch.tensor([h])

        kornia_sensor = K.geometry.camera.PinholeCamera(intr, extr, w, h)
        sensors_dict[sensor_id] = kornia_sensor
        
    return sensors_dict


/homeLocal/miniconda3/envs/moons/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pinhole_from_kapture(SAMPLE_DATASET)

{'kinect_rgb': <kornia.geometry.camera.pinhole.PinholeCamera at 0x7fccaee031f0>,
 'kinect_depth': <kornia.geometry.camera.pinhole.PinholeCamera at 0x7fccaee032e0>,
 'kinect_depth_reg': <kornia.geometry.camera.pinhole.PinholeCamera at 0x7fccaef73f40>}

Let's detect ORB keypoints and convert them to and from OpenCV

In [9]:
#hide
import nbdev
nbdev.export.notebook2script

AttributeError: module 'nbdev.export' has no attribute 'notebook2script'